In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import data
df = pd.read_csv('drive/MyDrive/Dataset_NA/RPData.csv')
# only did the first 30 rows b/c the whole dataframe took looooong to finish 
df = df.loc[:30]
df

,BeerID,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,PrimaryTemp
0,1,45.0,21.77,1.055,1.013,5.48,17.65,4.83,28.39,75,1.038,70.0,17.78
1,2,85.0,20.82,1.083,1.021,8.16,60.65,15.64,24.61,60,1.070,70.0,NaN
2,3,7.0,18.93,1.063,1.018,5.91,59.25,8.98,22.71,60,NaN,70.0,NaN
3,4,7.0,22.71,1.061,1.017,5.80,54.48,8.50,26.50,60,NaN,70.0,NaN
4,5,20.0,50.00,1.060,1.010,6.48,17.84,4.57,60.00,90,1.050,72.0,19.00
5,6,10.0,24.61,1.055,1.013,5.58,40.12,8.00,29.34,70,1.047,79.0,NaN
6,7,86.0,22.71,1.072,1.018,7.09,268.71,6.33,30.28,90,NaN,75.0,NaN
7,8,45.0,20.82,1.054,1.014,5.36,19.97,5.94,28.39,75,1.040,70.0,NaN
8,9,129.0,22.71,1.060,1.016,5.77,31.63,34.76,30.28,75,1.042,73.0,NaN
9,10,86.0,20.82,1.080,1.017,8.22,93.02,8.29,28.39,60,1.058,70.0,21.11


In [ ]:
columns = df.columns # get columns name

In [ ]:
ori_data = df.values

In [ ]:
attributes = ori_data.shape[1]
instances = ori_data.shape[0]

In [ ]:
null_indexes = np.argwhere(np.isnan(ori_data)) # row_col combinations who has null values

In [ ]:
def mean(dataframe):
    features = columns
    for col in features:
        avg = dataframe[col].mean()
        dataframe[col].replace(np.nan, avg, inplace = True)
    return dataframe

In [ ]:
def euclidean_distance(i,j): 
    column = len(i)
    distance = 0
    for a in range(column):
        if np.isnan(i[a]): # if there are nan in missing index then skip it
            continue
        distance = distance + (i[a]-j[a])**2
    return np.sqrt(distance)

In [ ]:
def sortSecond(val): 
    return val[1] # get value of 1st position

def knn(missingdata,k):
    fill_value = missingdata.copy()
    for j in null_indexes: # null_indexes contains all the row_col combinations of missing value
        distances = []
        for i in range(instances):
            if(i==j[0]): # if two rows are same then skip it
                continue
            else:
                if np.isnan(missingdata[i]).any(): # if row contains null anywhere then it skips
                    continue
                d = euclidean_distance(missingdata[j[0]], missingdata[i])  # distance only calculate for complete case
                distances.append([missingdata[i][j[1]], d] ) 
        distances.sort(key = sortSecond) # call sortSecond function and sort the list on the basis of value of 1st position 
        distances = np.array(distances)
        count = len(distances)
        nearest = [] # store the nearest neighbor in list
        for l in range(count):
            nearest.append(distances[l,0])    
        x = nearest[:k] #  slice list on the value of k
        mean = sum(x) / len(x)
        fill_value[j[0]][j[1]] = mean  # replace the nan with mean value  
    return fill_value

In [ ]:
knn5 = knn(ori_data, 5)
knn5_df = pd.DataFrame(knn5)
knn5_df.columns = columns
knn5_df

,BeerID,StyleID,Size(L),OG,FG,ABV,IBU,Color,BoilSize,BoilTime,BoilGravity,Efficiency,PrimaryTemp
0,1.0,45.0,21.770,1.055,1.013,5.48,17.65,4.83,28.390,75.0,1.0380,70.0,17.780
1,2.0,85.0,20.820,1.083,1.021,8.16,60.65,15.64,24.610,60.0,1.0700,70.0,18.780
2,3.0,7.0,18.930,1.063,1.018,5.91,59.25,8.98,22.710,60.0,1.0522,70.0,19.044
3,4.0,7.0,22.710,1.061,1.017,5.80,54.48,8.50,26.500,60.0,1.0522,70.0,19.044
4,5.0,20.0,50.000,1.060,1.010,6.48,17.84,4.57,60.000,90.0,1.0500,72.0,19.000
5,6.0,10.0,24.610,1.055,1.013,5.58,40.12,8.00,29.340,70.0,1.0470,79.0,19.044
6,7.0,86.0,22.710,1.072,1.018,7.09,268.71,6.33,30.280,90.0,1.0594,75.0,19.156
7,8.0,45.0,20.820,1.054,1.014,5.36,19.97,5.94,28.390,75.0,1.0400,70.0,18.578
8,9.0,129.0,22.710,1.060,1.016,5.77,31.63,34.76,30.280,75.0,1.0420,73.0,14.690
9,10.0,86.0,20.820,1.080,1.017,8.22,93.02,8.29,28.390,60.0,1.0580,70.0,21.110
